# Avoid Noisy Areas Using OpenRouteService Directions API

This notebook has built upon the work of Amandus, [link](https://openrouteservice.org/example-avoid-flooded-areas-with-ors/).

## Preprocessing

Inside the directory, run 'pip install -r requirements.txt' to download all the necessary packages.

Included in this repository is a CSV file with sample data based on outputs from a model analysing noise complaint data in Manhattan, New York. This notebook uses noise complaints classified as 'outside' as a proxy for whether a particular coordinate is noisy. 

Data Source: https://data.cityofnewyork.us/Social-Services/Noise-Complaints-in-2017-/be8n-q3nj/data

## Motivation

The directions endpoint of the OpenRouteService API can be used to receive the shortest or fastest route from point A to point B. This endpoint offers the avoid_polygons feature which is suited to avoid these areas as a primary objective and setting shortest path as the secondary objective. The data points are generally clustered so overlapping polygons are merged in order to reduce API calls to the ORS server.

Using this functionality, a routing service can be developed that avoids noisy areas. This can be especially important for people that are sensitive to these stimuli and wish to avoid them.

## Import Required Packages

In [59]:
import folium
from openrouteservice import client
import pandas as pd
from datetime import datetime
from shapely.geometry import Polygon, mapping, MultiPolygon, LineString, Point
from shapely.ops import unary_union
from pyproj import Transformer
import json
from shapely import affinity

In [60]:
# insert your ORS api key
api_key = ''
ors = client.Client(key=api_key)

## Data Inspection
Inspecting the data stored in the CSV. The important columns are extracted, the coordinate values, the hour of the day, whether the noise complaint is on a weekday or weekend and the 'noisyness' level designated Final Noise Complaint Category. 

Note:
* The noisyness level has been normalised
* The model outputs values for the weekend and weekday as binary [1,0]

In [61]:
df = pd.read_csv("../resources/data/noise_data/noisy_areas.csv", names=["Index","Created Date","Closed Date","Complaint Type","Descriptor","Location Type","Incident Zip","Incident Address","Street Name","Cross Street 1","Cross Street 2","Intersection Street 1","Intersection Street 2","Address Type","City","Resolution Action Updated Date","BBL","Borough","X Coordinate (State Plane)","Y Coordinate (State Plane)","Open Data Channel Type","Latitude","Longitude","Location","Hour","Month","Weekend","Weekday","Cluster","Noise Complaint Count","Noise Complaint Category","Final Noise Complaint Category"], skiprows=1)
df = df[['Longitude','Latitude','Hour','Weekday','Weekend',"Final Noise Complaint Category"]]

columns = df.columns

for column in columns:
    print("Column Name:", column, "\nData Type:", df[column].dtype, "\nNumber of Unique Values:", str(len(df[column].unique())))
    print(df[column].unique())
    print()

Column Name: Longitude 
Data Type: float64 
Number of Unique Values: 19484
[-73.92359    -73.97609401 -73.96705337 ... -73.95677104 -74.00344589
 -74.00153326]

Column Name: Latitude 
Data Type: float64 
Number of Unique Values: 19530
[40.86685084 40.77717631 40.76910729 ... 40.76986144 40.73353454
 40.72242106]

Column Name: Hour 
Data Type: int64 
Number of Unique Values: 24
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]

Column Name: Weekday 
Data Type: int64 
Number of Unique Values: 2
[0 1]

Column Name: Weekend 
Data Type: int64 
Number of Unique Values: 2
[1 0]

Column Name: Final Noise Complaint Category 
Data Type: float64 
Number of Unique Values: 25
[0.12 0.   0.04 0.08 0.36 0.16 0.28 0.24 0.44 0.8  0.2  0.96 0.68 0.52
 0.6  0.84 0.56 0.48 0.4  0.32 0.64 0.76 0.72 0.92 0.88]



Overview of data in the CSV

In [62]:
df.head(20)

,Longitude,Latitude,Hour,Weekday,Weekend,Final Noise Complaint Category
0,-73.923590,40.866851,0,0,1,0.12
1,-73.976094,40.777176,0,0,1,0.00
2,-73.967053,40.769107,0,0,1,0.00
3,-73.950344,40.794830,0,0,1,0.04
4,-73.930339,40.850211,0,0,1,0.00
5,-73.954211,40.823048,0,0,1,0.04
6,-73.975464,40.724447,0,0,1,0.00
7,-73.987215,40.736097,0,0,1,0.04
8,-73.962993,40.799079,1,0,1,0.12
9,-73.976181,40.735783,1,0,1,0.00


## Data Models

### Noise CSV

In [63]:
def predicted_locations(hour, day, count):
    # Read from csv
    df = pd.read_csv("../resources/data/noise_data/noisy_areas.csv", names=["Index","Created Date","Closed Date","Complaint Type","Descriptor","Location Type","Incident Zip","Incident Address","Street Name","Cross Street 1","Cross Street 2","Intersection Street 1","Intersection Street 2","Address Type","City","Resolution Action Updated Date","BBL","Borough","X Coordinate (State Plane)","Y Coordinate (State Plane)","Open Data Channel Type","Latitude","Longitude","Location","Hour","Month","Weekend","Weekday","Cluster","Noise Complaint Count","Noise Complaint Category","Final Noise Complaint Category"], skiprows=1)
    df = df[['Longitude','Latitude','Hour','Weekday','Weekend',"Final Noise Complaint Category"]]

    # Filter locations based on hour and day (0-4 - Weekday; 5-6 - Weekend)
    if 0 <= day <= 4:
        locations = df[(df["Hour"] == int(hour)) & (df["Weekday"] == 1) & (df["Final Noise Complaint Category"] >= count)]
    else:
        locations = df[(df["Hour"] == int(hour)) & (df["Weekend"] == 1) & (df["Final Noise Complaint Category"] >= count)]

    # Return coordinates of above the count percentage threshold
    response_list = []
    for _, location in locations.iterrows():
        response_dict = {
            'long': location["Longitude"],
            'lat': location["Latitude"],
        }
        response_list.append(response_dict)
    
    return response_list

## Setup Functions

Coordinates are represented as point geometries. A buffer is applied to each relevant coordinate to generate polygons. Street segments which overlap with a buffered tweet will be avoided. The upcoming function create_buffer_polygon transforms point geometries from WGS84 (EPSG: 4326) to UTM32N (EPSG: 32632), creates a 50 meter buffer around each one and transforms the geometries back to WGS84.

The create_route function requests routes from the OpenRouteService Directions API. Next to the regular input (starting point, end point, profile, preference), we pass avoid_polygons as an additional option.

The merge_intersecting_polygons function merges all overlapping polygons and returns these both the new merged polygons and the polygons that were not overlapping with other polygons.

In [64]:
# Function to create buffer around coordinate geometries and transform it to the needed coordinate system (WGS84)
def create_buffer_polygon(point_in, resolution=10, radius=50):
    transformer_wgs84_to_utm32n = Transformer.from_crs("EPSG:4326", "EPSG:3857")
    transformer_utm32n_to_wgs84 = Transformer.from_crs("EPSG:3857", "EPSG:4326")
    point_in_proj = transformer_wgs84_to_utm32n.transform(*point_in)
    point_buffer_proj = Point(point_in_proj).buffer(radius, resolution=resolution)
    
    # Adjust the aspect ratio of the buffer to make it more of a circle than an oval shape
    transformed_buffer = affinity.scale(point_buffer_proj, xfact=1, yfact=5)
    
    # Transform back to WGS84
    poly_wgs = [transformer_utm32n_to_wgs84.transform(*point) for point in transformed_buffer.exterior.coords]
    return poly_wgs


# Function to request directions with avoided_polygon feature
def create_route(data, avoided_point_list):
    route_request = {'coordinates': data,
                        'format': 'geojson',
                        'profile': 'foot-walking',
                        'preference': 'shortest',
                        'options': {'avoid_polygons': mapping(MultiPolygon(avoided_point_list))}}
    route_directions = ors.directions(**route_request)
    return route_directions


# Function to create buffer around requested route
def create_buffer(r_directions):
    coordinates = r_directions['features'][0]['geometry']['coordinates']
    linestring = LineString(coordinates)
    dilated_route = linestring.buffer(0.0005)
    return dilated_route

# Function to style data
def style_function(color):
    return lambda feature: dict(color=color)

# Function to merge intersecting polygons, loops through every polygon to check for overlaps 
# and returns all merged polygons
def merge_intersecting_polygons(poly_list):
    merged_polygons = []
    for poly in poly_list:
        if not merged_polygons:
            merged_polygons.append(poly)
        else:
            new_merged = []
            for merged_poly in merged_polygons:
                if poly.intersects(merged_poly):
                    poly = poly.union(merged_poly)
                else:
                    new_merged.append(merged_poly)
            new_merged.append(poly)
            merged_polygons = new_merged
    return merged_polygons

## Analysis

Creates a visualisation of the noisy areas on the map for a given day and time. To get an overview of the noisy areas all the data is loaded for the current time and area. The CreateBufferPolygon function is used to create a buffer polygons around the points classified as noisy.

In [65]:
# Create map
map = folium.Map(location=[40.715709, -74.005092], zoom_start=14)

# Time and date initialised to current time and date
now = datetime.now()
prediction_hour = int(now.strftime("%H"))
prediction_day = int(now.weekday())

# Retrieve top 50% of noisy places according to the output of the models
noise_threshold = 0.5

# Get the top 50% of places classified as noisy for prescribed time and date
all_locations = predicted_locations(prediction_hour, prediction_day, noise_threshold)

point_geometry = []

# Loop through noisy coordinates and place a polygon around them
for location in all_locations:
    position = [location['long'], location['lat']]
    point_buffer = create_buffer_polygon(position)
    point_geometry.append(Polygon(point_buffer))

# Join clustering polygons together for folium display
union_poly = unary_union(point_geometry)
union_poly_geojson = mapping(union_poly)

# Convert to GeoJSON-compatible format
union_poly_json = json.dumps(union_poly_geojson)
union_poly_data = json.loads(union_poly_json)

# Display noisy areas on folium map
folium.features.GeoJson(data=union_poly_data,
                        name='Noisy Areas',
                        style_function=style_function('#ffd699')).add_to(map)

map.add_child(folium.map.LayerControl())
map

## Routing
Visualises the routing that takes place for a particular time and day. In the beginning two functions, CreateRoute and CreateBuffer were created. The CreateRoute function requests the shortest route from A to B for the foot-walking profile. It also avoids the areas which are included in the avoided_point_list. This list is empty in the beginning.

To retrieve a route that does not intersect with any noisy areas, the merge_intersecting_polygons function is used to merge all intersecting polygons. These polygons are sent to the API as areas to avoid, it then returns back the avoidance route for rendering. By merging the polygons together, instead of sending polygons for every coordinate, the package size sent to the API is much smaller. It can only handle a limited number of poylgons as areas to avoid currently.

In [66]:
# Create map
map = folium.Map(location=[40.721954, -74.008310], zoom_start=16)

# Coordinate that demonstrates re-routing
coordinates = [[-74.008627, 40.719967], [-74.007763, 40.724708]]

# Starting location marker (Red)
folium.map.Marker(
    location=list(reversed(coordinates[0])),
    icon=folium.Icon(color="red"),
    ).add_to(map)

# Destination marker (Blue)
folium.map.Marker(
    location=list(reversed(coordinates[1])),
    icon=folium.Icon(color="blue"),
    ).add_to(map)

# Time and date can be initialised to the current time and date
# now = datetime.now()
# prediction_hour = int(now.strftime("%H"))
# prediction_day = int(now.weekday())

# Time and date initialised to Monday at 17:00 to demonstrate re-routing
prediction_hour = "17"
prediction_day = 0

# Retrieve top 50% of noisy places according to the output of the models
noise_threshold = 0.5

# Get the top 50% of places classified as noisy for prescribed time and date
all_locations = predicted_locations(prediction_hour, prediction_day, noise_threshold)

# List of areas to avoid, for merging for and sending to the API and for the folium map for display
avoid_areas_ls = []
point_geometry = []

# Loop through noisy coordinates and place a polygon around them
for location in all_locations:
    position = [location['long'], location['lat']]
    point_buffer = Polygon(create_buffer_polygon(position))
    avoid_areas_ls.append(point_buffer)
    point_geometry.append(Polygon(point_buffer))

# Create a buffer around the start location and destination
start_buffer = Polygon(create_buffer_polygon(coordinates[0]))
end_buffer = Polygon(create_buffer_polygon(coordinates[1]))

# Join clustering polygons together for folium display
union_poly = unary_union(point_geometry)
union_poly_geojson = mapping(union_poly)

# Convert to GeoJSON-compatible format
union_poly_json = json.dumps(union_poly_geojson)
union_poly_data = json.loads(union_poly_json)

# Display noisy areas on folium map
folium.features.GeoJson(data=union_poly_data,
                        name='Busy Areas',
                        style_function=style_function('#ffd699')).add_to(map)

try:
    # Create regular route with an empty list of areas to avoid
    optimal_route = create_route(coordinates, [])

    # Plot optimal route
    folium.features.GeoJson(
                        data=optimal_route,
                        name='Regular Route',
                        style_function=style_function('#ff5050'), overlay=True).add_to(map)

    print('Generated regular route.')

except Exception as e:
    print(e)

# Create buffer around optimal route
avoidance_directions = create_buffer(optimal_route)

# Initialise avoidance_route as an empty string
avoidance_route = ""

# Merge intersecting polygons
intersecting_polygons = merge_intersecting_polygons(avoid_areas_ls)

# Create a copy of the list of intersecting polygons and remove the starting location and destination, otherwise the
# route can't start or end at the correct locations
intersecting_polygons_copy = intersecting_polygons.copy()
for poly in intersecting_polygons_copy:
    if poly.intersects(start_buffer) or poly.intersects(end_buffer):
        intersecting_polygons.remove(poly)

try:
    print("Number Unmerged Polygons/ Unmerged Noisy Coordinates: " + str(len(avoid_areas_ls)))
    print("Total Merged Polygons/ Merged Noisy Areas: " + str(len(intersecting_polygons)))
    avoidance_route = create_route(coordinates, intersecting_polygons)
except Exception as e:
    print(e)

if avoidance_route != optimal_route:
    # Display the avoidance route
    folium.features.GeoJson(
                    data=avoidance_route,
                    name='Alternative Route',
                    style_function=style_function('#006600'),
                    overlay=True).add_to(map)
    print('Generated alternative route, which avoids affected areas.')

map.add_child(folium.map.LayerControl())
map

Generated regular route.
Number Unmerged Polygons/ Unmerged Noisy Coordinates: 371
Total Merged Polygons/ Merged Noisy Areas: 17
Generated alternative route, which avoids affected areas.


## Export Map Views

The following exports an interactive HTML of a map for every day and time possible for inspection. This helps to determine the correct threshold that defines an area as noisy, in the above it's the top 50%. With too high a threshold, the clusters become too large to be practically routed around and too small, the user could be routed through a noisy area.

Note: This can take a while few minutes run as it runs for 48 iterations and has to merge all the coordinate level poylgons together to render them on the map

### Noisy Areas

In [67]:
# Create a loop for both a weekday (0 = Monday) & weekend (6 = Sunday)
for prediction_day in [0, 6]:
    # Create a map for every hour in a day (0-24)
    for prediction_hour in range(24):
        # Create map
        map = folium.Map(location=[40.76657321777155, -73.9831392189498], zoom_start=12)

        # Retrieve top 50% of noisy places according to the output of the models
        noise_threshold = 0.5
        
        # Get the top 50% of places classified as noisy for prescribed time and date
        all_locations = predicted_locations(str(prediction_hour), prediction_day, noise_threshold)

        print("Day: " + str(prediction_day) + " Time: " + str(prediction_hour))
        print("Number of Unmerged Noisy Areas: " + str(len(all_locations)))
        
        point_geometry = []

        # Loop through noisy coordinates and place a polygon around them
        for location in all_locations:
            position = [location['long'], location['lat']]
            point_buffer = create_buffer_polygon(position)
            point_geometry.append(Polygon(point_buffer))

        # Join clustering polygons together for folium display
        union_poly = unary_union(point_geometry)
        union_poly_geojson = mapping(union_poly)

        # Convert to GeoJSON-compatible format
        union_poly_json = json.dumps(union_poly_geojson)
        union_poly_data = json.loads(union_poly_json)

        # Display noisy areas on folium map
        folium.features.GeoJson(data=union_poly_data,
                                name='Noisy Areas',
                                style_function=style_function('#ffd699')).add_to(map)

        # Save the map as a HTML doc in the map_export subdirectory
        map.save(f'../resources/img/map_exports/map_noise_{prediction_day}_{prediction_hour}.html')


Day: 0 Time: 0
Number of Unmerged Noisy Areas: 2072
Day: 0 Time: 1
Number of Unmerged Noisy Areas: 1159
Day: 0 Time: 2
Number of Unmerged Noisy Areas: 437
Day: 0 Time: 3
Number of Unmerged Noisy Areas: 173
Day: 0 Time: 4
Number of Unmerged Noisy Areas: 167
Day: 0 Time: 5
Number of Unmerged Noisy Areas: 98
Day: 0 Time: 6
Number of Unmerged Noisy Areas: 360
Day: 0 Time: 7
Number of Unmerged Noisy Areas: 576
Day: 0 Time: 8
Number of Unmerged Noisy Areas: 296
Day: 0 Time: 9
Number of Unmerged Noisy Areas: 449
Day: 0 Time: 10
Number of Unmerged Noisy Areas: 241
Day: 0 Time: 11
Number of Unmerged Noisy Areas: 175
Day: 0 Time: 12
Number of Unmerged Noisy Areas: 130
Day: 0 Time: 13
Number of Unmerged Noisy Areas: 143
Day: 0 Time: 14
Number of Unmerged Noisy Areas: 246
Day: 0 Time: 15
Number of Unmerged Noisy Areas: 261
Day: 0 Time: 16
Number of Unmerged Noisy Areas: 301
Day: 0 Time: 17
Number of Unmerged Noisy Areas: 371
Day: 0 Time: 18
Number of Unmerged Noisy Areas: 532
Day: 0 Time: 19
Numbe

## Conclusion
A routing service that can avoid particular areas specified by the user allows for a greater diversity of use. This type of service empowers people to find optimal pathways based on additional criteria to shortest path. For people that are sensitive to a range of stimuli, the shortest path may not be their optimal path. In this example, noisy areas are avoided in Manhattan, New York but the same concept could be expanded to optimise the user's route based on other criteria.

To tackle this issue, this notebook shows how the direction feature avoid_polygons of the OpenRouteService API can be used. It allows avoiding certain areas (e.g. noisy areas) and to request the fastest or shortest route for different routes while walking. This notebook has built upon the work of Amandus, [link](https://openrouteservice.org/example-avoid-flooded-areas-with-ors/).